# Application with random weights (Genetic Algo)

In [10]:
# Lire et analyser le fichier de dataset
file_path = "Data\pi-13-1000-1000-001.kna"

# Afficher un aperçu du contenu du fichier
with open(file_path, "r") as file:
    dataset_preview = file.readlines()[:20]  # Lire les 20 premières lignes pour analyse

dataset_preview


<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Epulapp\AppData\Local\Temp\ipykernel_16532\3412950412.py:2: SyntaxWarning: invalid escape sequence '\p'
  file_path = "Data\pi-13-1000-1000-001.kna"


['NAME: pi-13-1000-1000-001.kna\n',
 'COMMENT: strongly correlated span(2 10)\n',
 'PROBLEM: knapsack\n',
 'NB_ITEMS: 1000\n',
 'MAX_CAPACITY: 3177\n',
 '\n',
 'DATA [id profit weight]:\n',
 '1 234 114\n',
 '2 39 19\n',
 '3 1053 873\n',
 '4 351 291\n',
 '5 585 485\n',
 '6 78 38\n',
 '7 117 97\n',
 '8 234 194\n',
 '9 312 152\n',
 '10 156 76\n',
 '11 156 76\n',
 '12 273 133\n',
 '13 351 291\n']

In [ ]:
import random

# Paramètres de l'algorithme génétique
population_size = 50
num_generations = 20
mutation_rate = 0.9

# Fonction pour charger les données depuis un fichier

def load_knapsack_data(file_path):
    weights = []
    profits = []
    max_capacity = 0
    
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue

        if parts[0] == "MAX_CAPACITY:":
            max_capacity = int(parts[1])
        elif parts[0].isdigit():  # Ligne contenant les données des objets
            _, profit, weight = map(int, parts)  # Ignorer l'ID et extraire profit et poids
            profits.append(profit)
            weights.append(weight)
    
    return weights, profits, max_capacity

# Charger les données depuis le fichier
file_path = "Data/pi-12-1000-1000-001.kna"  # Remplacer par le bon chemin
weights, profits, max_capacity = load_knapsack_data(file_path)

# Classe représentant une solution pour le problème du sac à dos
class KnapsackSolution:
    def __init__(self, weights, profits, max_capacity, chromosome=None):
        self.weights = weights
        self.profits = profits
        self.capacity = max_capacity
        self.num_items = len(weights)

        # Générer un chromosome valide dès l'initialisation
        if chromosome is None:
            self.chromosome = self.generate_valid_chromosome()
        else:
            self.chromosome = chromosome

        self.total_weight = sum(w for w, c in zip(self.weights, self.chromosome) if c)
        self.total_profit = sum(p for p, c in zip(self.profits, self.chromosome) if c)

        # Correction des solutions non valides
        self.correct_solution()

    def generate_valid_chromosome(self):
        """Génère un chromosome qui respecte la contrainte de poids."""
        chromosome = [False] * self.num_items
        indices = list(range(self.num_items))
        random.shuffle(indices)
        current_weight = 0

        for i in indices:
            if current_weight + self.weights[i] <= self.capacity:
                chromosome[i] = True
                current_weight += self.weights[i]

        return chromosome

    def correct_solution(self):
        """Si la solution dépasse la capacité, retire des objets au hasard."""
        while self.total_weight > self.capacity:
            indices = [i for i, c in enumerate(self.chromosome) if c]
            if not indices:
                break
            remove_idx = random.choice(indices)
            self.chromosome[remove_idx] = False
            self.total_weight -= self.weights[remove_idx]
            self.total_profit -= self.profits[remove_idx]

    def __lt__(self, other):
        return self.total_profit > other.total_profit  # Tri en ordre décroissant de profit

# Sélection par tournoi
def tournament_selection(population, k=5):
    selected = random.sample(population, k)
    return max(selected, key=lambda x: x.total_profit)

# Croisement (one-point crossover)
def crossover(parent1, parent2, max_capacity):
    point = random.randint(1, len(parent1.chromosome) - 1)
    child1 = parent1.chromosome[:point] + parent2.chromosome[point:]
    child2 = parent2.chromosome[:point] + parent1.chromosome[point:]
    return KnapsackSolution(weights, profits, max_capacity, child1), KnapsackSolution(weights, profits, max_capacity, child2)

# Mutation (bit-flip mutation)
def mutate(solution, max_capacity):
    new_chromosome = solution.chromosome[:]
    for i in range(len(new_chromosome)):
        if random.random() < mutation_rate:
            new_chromosome[i] = not new_chromosome[i]  # Inverser le bit
    return KnapsackSolution(weights, profits, max_capacity, new_chromosome)

# Initialisation de la population
population = [KnapsackSolution(weights, profits, max_capacity) for _ in range(population_size)]

# Boucle principale de l'algorithme génétique
for generation in range(num_generations):
    population.sort()  # Trier les solutions par profit décroissant

    # Affichage du meilleur individu de la génération actuelle
    best_solution = population[0]
    print(f"Gen {generation}: Best Profit = {best_solution.total_profit}, Weight = {best_solution.total_weight}")

    # Nouvelle génération
    new_population = population[:10]  # Garder les 10 meilleures solutions (élitisme)
    
    while len(new_population) < population_size:
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)
        child1, child2 = crossover(parent1, parent2, max_capacity)
        new_population.append(mutate(child1, max_capacity))
        if len(new_population) < population_size:
            new_population.append(mutate(child2, max_capacity))

    population = new_population

# Affichage du meilleur résultat final
best_solution = max(population, key=lambda x: x.total_profit)
print(f"\nFinal Best Solution: Profit = {best_solution.total_profit}, Weight = {best_solution.total_weight}")


Gen 0: Best Profit = 4469, Weight = 4497
Gen 1: Best Profit = 4469, Weight = 4497
Gen 2: Best Profit = 4469, Weight = 4497
Gen 3: Best Profit = 4469, Weight = 4497
Gen 4: Best Profit = 4469, Weight = 4497
Gen 5: Best Profit = 4469, Weight = 4497
Gen 6: Best Profit = 4469, Weight = 4497
Gen 7: Best Profit = 4469, Weight = 4497
Gen 8: Best Profit = 4469, Weight = 4497
Gen 9: Best Profit = 4469, Weight = 4497
Gen 10: Best Profit = 4469, Weight = 4497
Gen 11: Best Profit = 4469, Weight = 4497
Gen 12: Best Profit = 4514, Weight = 4528
Gen 13: Best Profit = 4514, Weight = 4528
Gen 14: Best Profit = 4514, Weight = 4528


# Méthode Tabou